# Output Factor Calibration

This script computes field-size dependent output corrections that account for variations not captured by head scatter alone.

**What you need:**
- Output factors for field sizes from 4×4 to 40×40 cm
- All measurements normalized to 10×10 cm = 1.000

**What to do:**
1. Enter your field sizes in `FIELD_SIZES` (in mm)
2. Enter measured output factors in `OF_MEASUREMENTS` (same as in calibration step 2.)
3. Run to compute correction factors
4. Verify the plot shows good agreement

**Output:**
- `output_factors` array [[field_sizes_mm], [correction_factors]]


In [ ]:
from sympy import false
import torch
import numpy as np
import matplotlib.pyplot as plt
import math
import pydicom
from pydose_rt import DoseEngine
from pydose_rt.data import MachineConfig, Phantom, loaders, Beam

# ============================================
# USER INPUTS
# ============================================
FIELD_SIZES = [40, 50, 70, 80, 100, 120, 150, 200, 250, 300, 350, 400]  # mm
OF_MEASUREMENTS = [     0.884,     0.912,     0.955,     0.971,    1.000,    1.021,    1.045,    1.075,    1.096,    1.111,    1.123,    1.130 ]
CALIBRATION_MU = 110


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float32


kernel_size = 101
results = []

machine_config = MachineConfig(
    preset="../../src/pydose_rt/data/machine_presets/umea_10MV.json",
    output_factors=None
)
resolution = (4.0, 4.0, 4.0)
ct_array_shape = (100, 100, 100)
phantom = Phantom.from_uniform_water(shape=ct_array_shape, spacing=resolution).to(device).to(dtype)
modelled = []
beam_template = Beam.create(
    gantry_angle_deg=0.0, 
    number_of_leaf_pairs=60, 
    collimator_angle_deg=0.0, 
    field_size_mm=(100, 100), 
    iso_center=(200, 100, 200), 
    device=device, 
    dtype=dtype)
dose_engine = DoseEngine(
    machine_config, 
    kernel_size,
    dose_grid_spacing=phantom.resolution,
    dose_grid_shape=phantom.density_image.shape,
    beam_template=beam_template,
    device=device,
    dtype=dtype,
    adjust_values=False
)
dose_engine.calibrate(110)
for field_size in FIELD_SIZES:
    number_of_beams = 1
    starting_angle = 0

    beam = Beam.create(
        gantry_angle_deg=0.0, 
        number_of_leaf_pairs=60, 
        collimator_angle_deg=0.0, 
        field_size_mm=(field_size, field_size), 
        iso_center=(200, 100, 200), 
        device=device, 
        dtype=dtype)
    beam.mu = CALIBRATION_MU * beam.mu

    dose = dose_engine.compute_dose(
        beam,
        density_image=phantom.density_image).detach()
    modelled.append(dose[0, *dose_engine.iso_center_voxel].item())


plt.plot(FIELD_SIZES, modelled, label=f"Model", marker='o')
plt.plot(FIELD_SIZES, OF_MEASUREMENTS, label=f"Measurement", marker='x')
plt.legend()
plt.show()

corrections = np.array(OF_MEASUREMENTS) / np.array(modelled)
corrections = corrections / corrections[4]

profiles = np.round(np.stack([FIELD_SIZES, corrections], 0), 4)

def fmt(x):
    s = np.format_float_positional(x, trim='-')
    return s if '.' in s else s + '.0'

formatted = np.array2string(
    profiles,
    formatter={'float_kind': fmt},
    separator=', '       # ← comma between values
)

print(formatted)